<a href="https://colab.research.google.com/github/Staffilon/KestraDataOrchestrator/blob/main/Analysis_and_Exploration/merged_pieces_and_energy_creation_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd "/content/gdrive/MyDrive/Dati-TBDM/refined_data"

/content/gdrive/.shortcut-targets-by-id/1xGTanGyuk-TCLCMoGz5VNG7sUEscVC-Z/Dati-TBDM/refined_data


In [42]:
import pandas as pd
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count
import numpy as np
import multiprocessing

# Importing the necessary datasets

Import the Energy dataset (not the one merged with stop codes)

In [29]:
energy_df = pd.read_csv('modified_energy_data.csv', encoding='latin1')

Import the energy dataset merged with the stop codes

In [30]:
stop_df = pd.read_csv('energy_fermate_merged.csv', encoding = 'latin1')

Import the produced pieces dataset

In [31]:
pieces_df = pd.read_csv('produced_pieces_dataset.csv', encoding='latin1')

In [32]:
pieces_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552095 entries, 0 to 7552094
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 int64  
 1   COD_MACC           float64
 2   COD_ART            object 
 3   TIMESTAMP_INIZIO   object 
 4   TIMESTAMP_FINE     object 
 5   NUMERO_PEZZI_PROD  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 345.7+ MB


# Cleaning out the produced pieces dataset

Taking out the machine that are not present in the energy dataset

In [33]:
pieces_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552095 entries, 0 to 7552094
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 int64  
 1   COD_MACC           float64
 2   COD_ART            object 
 3   TIMESTAMP_INIZIO   object 
 4   TIMESTAMP_FINE     object 
 5   NUMERO_PEZZI_PROD  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 345.7+ MB


In [34]:
# Extract unique machines from energy_df
unique_machines_energy = energy_df['channel'].unique()

In [35]:
# Filter pieces_df to include only rows where COD_MACC is present in unique_machines_energy
pieces_df = pieces_df[pieces_df['COD_MACC'].isin(unique_machines_energy)]

In [36]:
pieces_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1546520 entries, 210 to 7551634
Data columns (total 6 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ID                 1546520 non-null  int64  
 1   COD_MACC           1546520 non-null  float64
 2   COD_ART            1546520 non-null  object 
 3   TIMESTAMP_INIZIO   1546520 non-null  object 
 4   TIMESTAMP_FINE     1546520 non-null  object 
 5   NUMERO_PEZZI_PROD  1546520 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 82.6+ MB


Taking out the dates that are not present in the energy dataset

In [37]:
# Convert TIMESTAMP_INIZIO and TIMESTAMP_FINE columns to datetime objects
pieces_df['TIMESTAMP_INIZIO'] = pd.to_datetime(pieces_df['TIMESTAMP_INIZIO']).dt.tz_localize(None)
pieces_df['TIMESTAMP_FINE'] = pd.to_datetime(pieces_df['TIMESTAMP_FINE']).dt.tz_localize(None)

# Convert 'TimeStamp' column in energy_df to datetime object
energy_df['TimeStamp'] = pd.to_datetime(energy_df['TimeStamp']).dt.tz_localize(None)

In [38]:
# Find unique dates present in energy_df
unique_dates_energy = energy_df['TimeStamp'].dt.date.unique()

In [39]:
# Filter pieces_df based on whether TIMESTAMP_INIZIO date is in unique_dates_energy
pieces_df = pieces_df[pieces_df['TIMESTAMP_INIZIO'].dt.date.isin(unique_dates_energy)]

In [40]:
pieces_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 715058 entries, 210 to 6808758
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ID                 715058 non-null  int64         
 1   COD_MACC           715058 non-null  float64       
 2   COD_ART            715058 non-null  object        
 3   TIMESTAMP_INIZIO   715058 non-null  datetime64[ns]
 4   TIMESTAMP_FINE     715058 non-null  datetime64[ns]
 5   NUMERO_PEZZI_PROD  715058 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 38.2+ MB


# Mixing up the pieces and energy datasets

In [ ]:
def calculate_energy_value(row):
    start_time = row['TIMESTAMP_INIZIO']
    end_time = row['TIMESTAMP_FINE']
    machine_code = row['COD_MACC']

    matching_energy_rows = energy_df[(energy_df['channel'] == machine_code) &
                                     (energy_df['TimeStamp'] >= start_time) &
                                     (energy_df['TimeStamp'] <= end_time)]

    if not matching_energy_rows.empty:
        return matching_energy_rows['Ea_Imp'].mean()
    else:
        closest_energy_value = energy_df[(energy_df['channel'] == machine_code) &
                                         (energy_df['TimeStamp'] >= start_time - timedelta(minutes=15)) &
                                         (energy_df['TimeStamp'] <= start_time)]

        if not closest_energy_value.empty:
            return closest_energy_value['Ea_Imp'].mean()
        else:
            return None

def calculate_energy_values_chunk(chunk):
    return chunk.apply(calculate_energy_value, axis=1)

if __name__ == '__main__':
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=num_processes)

    chunk_size = len(pieces_df) // num_processes
    chunks = [pieces_df.iloc[i:i+chunk_size] for i in range(0, len(pieces_df), chunk_size)]

    results = pool.map(calculate_energy_values_chunk, chunks)
    pieces_df['energy_value'] = pd.concat(results)

    pieces_df.to_csv('updated_produced_pieces_dataset.csv', index=False)


In [37]:
updated_pieces_df = pd.read_csv('updated_produced_pieces_dataset.csv')

In [38]:
# Check the number of non-null values in the 'energy_value' column
energy_value_count = updated_pieces_df['energy_value'].count()

print("Number of non-null values in the 'energy_value' column:", energy_value_count)

Number of non-null values in the 'energy_value' column: 0


In [39]:
# Preprocess energy data to create a lookup table
energy_lookup = energy_df.set_index('TimeStamp')['Ea_Imp']

# Print the energy_lookup table to check its contents
print(energy_lookup)

TimeStamp
2022-08-10 00:00:00    113.7
2022-08-10 00:15:00    117.1
2022-08-10 00:30:00    110.8
2022-08-10 00:45:00    111.2
2022-08-10 01:00:00    122.5
                       ...  
2023-08-22 21:30:00      0.0
2023-08-22 21:45:00      0.0
2023-08-22 22:00:00      0.0
2023-08-22 22:15:00      0.0
2023-08-22 22:30:00      0.0
Name: Ea_Imp, Length: 567805, dtype: float64
